In [1]:
#default_exp filter

In [2]:
#hide
import sys
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# Filtragem
> Este módulo concentra funções auxiliares específicas que filtram os dados do banco com campos e formatação de interesse para aplicações específicas como o [appAnalise](https://github.com/EricMagalhaesDelgado/appAnalise) por exemplo.

In [3]:
#export
import os
from pathlib import Path
import json
import pandas as pd
from datetime import datetime
from openpyxl import load_workbook
from anateldb.query import *
from anateldb.constants import console, APP_ANALISE
from fastcore.test import *
from fastcore.script import call_parse, Param, store_true
from pyarrow import ArrowInvalid
from geopy.distance import geodesic
from rich import print

In [4]:
#export
def bump_version(version, part=2):
    version = version.split(".")
    version[part] = str(int(version[part]) + 1)
    for i in range(part + 1, 3):
        version[i] = "0"
    return ".".join(version)

In [5]:
#exporti
def append_df_to_excel(
    filename,
    df,
    sheet_name="Sheet1",
    startrow=None,
    truncate_sheet=False,
    **to_excel_kwargs,
):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    @param filename: File path or existing ExcelWriter
                     (Example: '/path/to/file.xlsx')
    @param df: DataFrame to save to workbook
    @param sheet_name: Name of sheet which will contain DataFrame.
                       (default: 'Sheet1')
    @param startrow: upper left cell row to dump data frame.
                     Per default (startrow=None) calculate the last row
                     in the existing DF and write to the next row...
    @param truncate_sheet: truncate (remove and recreate) [sheet_name]
                           before writing DataFrame to Excel file
    @param to_excel_kwargs: arguments which will be passed to `DataFrame.to_excel()`
                            [can be a dictionary]
    @return: None

    Usage examples:

    >>> append_df_to_excel('d:/temp/test.xlsx', df)

    >>> append_df_to_excel('d:/temp/test.xlsx', df, header=None, index=False)

    >>> append_df_to_excel('d:/temp/test.xlsx', df, sheet_name='Sheet2',
                           index=False)

    >>> append_df_to_excel('d:/temp/test.xlsx', df, sheet_name='Sheet2',
                           index=False, startrow=25)

    (c) [MaxU](https://stackoverflow.com/users/5741205/maxu?tab=profile)
    """
    # Excel file doesn't exist - saving and exiting
    if not Path(filename).is_file():
        df.to_excel(
            filename,
            sheet_name=sheet_name,
            startrow=startrow if startrow is not None else 0,
            **to_excel_kwargs,
        )
        return

    # ignore [engine] parameter if it was passed
    if "engine" in to_excel_kwargs:
        to_excel_kwargs.pop("engine")

    writer = pd.ExcelWriter(
        filename, engine="openpyxl", mode="a", if_sheet_exists="replace"
    )

    # try to open an existing workbook
    writer.book = load_workbook(filename)

    # get the last row in the existing Excel sheet
    # if it was not specified explicitly
    if startrow is None and sheet_name in writer.book.sheetnames:
        startrow = writer.book[sheet_name].max_row

    # truncate sheet
    if truncate_sheet and sheet_name in writer.book.sheetnames:
        # index of [sheet_name] sheet
        idx = writer.book.sheetnames.index(sheet_name)
        # remove [sheet_name]
        writer.book.remove(writer.book.worksheets[idx])
        # create an empty sheet [sheet_name] using old index
        writer.book.create_sheet(sheet_name, idx)

    # copy existing sheets
    writer.sheets = {ws.title: ws for ws in writer.book.worksheets}

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)

    # save the workbook
    writer.save()

In [6]:
#export
@call_parse
def formatar_db(
    path: Param("Pasta onde salvar os arquivos", str),
    up_stel: Param("Atualizar a base do Stel", store_true) = False,
    up_radcom: Param("Atualizar a base do Radcom", store_true) = False,
    up_mosaico: Param("Atualizar a base do Mosaico", store_true) = False,
    up_icao: Param("Atualizar a base do ICAO", store_true) = False,
    up_pmec: Param("Atualizar a base do PMEC", store_true) = False,
    up_geo: Param("Atualizar a base do Geo", store_true) = False,
) -> None:
    dest = Path(path)
    dest.mkdir(parents=True, exist_ok=True)
    time = datetime.today().strftime("%d/%m/%Y %H:%M:%S")
    console.print(":scroll:[green]Lendo as bases de dados...")
    rd = read_base(path, up_stel, up_radcom, up_mosaico, up_icao)
    rd["Validade_RF"] = rd.Validade_RF.astype("string").fillna("")
    rd["Data_Ato"] = rd.Data_Ato.astype("string").fillna("")
    rd["Status"] = rd.Status.astype("string")
    rd["Classe"] = rd.Classe.astype("string")
    rd.loc[rd["Classe"].notna(), "Status"] = (
        rd.loc[rd["Classe"].notna(), "Status"]
        + ", "
        + rd.loc[rd["Classe"].notna(), "Classe"]
    )
    # rd.loc[rd["Status"].isna(), "Status"] = rd.loc[
    #     rd["Status"].isna(), "Num_Serviço"
    # ].astype("string")

    rd["Descrição"] = (
        "["
        + rd.Fonte.astype("string")
        + "] "
        + rd.Status.astype("string").fillna("-")
        + ", "
        # + rd.Classe.astype("string").fillna("-")
        # + ", "
        + rd.Entidade.astype("string").fillna("-").str.title()
        + " ("
        + rd.Fistel.astype("string").fillna("-")
        + ", "
        + rd["Número_da_Estação"].astype("string").fillna("-")
        + "), "
        + rd.Município.astype("string").fillna("-")
        + "/"
        + rd.UF.astype("string").fillna("-")
    )

    export_columns = [
        "Frequência",
        "Latitude",
        "Longitude",
        "Descrição",
        "Num_Serviço",
        "Número_da_Estação",
        "Classe_Emissão",
        "Largura_Emissão",
    ]
    rd = rd.loc[:, export_columns]
    rd.columns = APP_ANALISE
    common, new = read_aero(path, up_icao, up_pmec, up_geo)
    rd = merge_aero(rd, common, new)
    rd = df_optimize(rd, exclude=["Frequency"])
    console.print(":card_file_box:[green]Salvando os arquivos...")
    d = json.loads((dest / "VersionFile.json").read_text())
    try:
        cache = pd.read_feather(f"{dest}/AnatelDB.fth")
    except (ArrowInvalid, FileNotFoundError):
        cache = pd.DataFrame()

    if not rd.equals(cache):
        console.print(
            ":new: [green] A base de dados mudou desde a última atualização! Salvando o novo arquivo e atualizando a versão"
        )
        date = pd.DataFrame(columns=[time])
        try:
            rd.to_feather(Path(f"{dest}/AnatelDB.fth").open("bw"))
        except ArrowInvalid:
            Path(f"{dest}/AnatelDB.fth").unlink()
        with pd.ExcelWriter(f"{dest}/AnatelDB.xlsx", engine="xlsxwriter") as workbook:
            date.to_excel(workbook, sheet_name="ExtractDate", index=False)
            rd.to_excel(workbook, sheet_name="DataBase", index=False)
        d["anateldb"]["Version"] = bump_version(d["anateldb"]["Version"])
    else:
        console.print(
            ":recycle: [green] A base de dados não mudou desde a última atualização, a versão não será atualizada, somente a data de verificação"
        )

    console.print("Sucesso :zap:")
    d["anateldb"]["ReleaseDate"] = datetime.today().strftime("%d/%m/%Y")
    json.dump(d, (dest / "VersionFile.json").open("w"))

In [7]:
pasta = Path('c:/Users/rsilva/db')

In [8]:
formatar_db(pasta)

📜Lendo as bases de dados...

🗃Salvando os arquivos...

🆕  A base de dados mudou desde a última atualização! Salvando o novo arquivo e atualizando a
versão

Sucesso ⚡

In [9]:
if not pasta.exists():
    import subprocess
    result = subprocess.run(['git', 'clone', 'https://github.com/ronaldokun/AnatelDatabase.git'])
    pasta = Path.cwd() / 'AnatelDatabase'

In [10]:
df = pd.read_feather(f'{pasta}/AnatelDB.fth') #, engine='openpyxl', sheet_name='DataBase')

In [11]:
df.tail()

,Frequency,Latitude,Longitude,Description,Service,Station,Class,BW
885511,85469.0,-16.592699,-49.267799,"[STEL] L, FX, Tim S A (50417425295, 1008754061...",19,1008754061,Q7W,62M5
885512,85469.0,-1.358931,-48.385670,"[STEL] L, FX, Tim S A (50417425295, 1009786951...",19,1009786951,Q7W,750M
885513,85469.0,-9.937445,-67.827751,"[STEL] L, FX, Tim S A (50417425295, 1005059940...",19,1005059940,Q7W,62M5
885514,85469.0,-3.734861,-38.462749,"[STEL] L, FX, Tim S A (50417425295, 1008775875...",19,1008775875,Q7W,62M5
885515,85469.0,-8.266858,-36.006023,"[STEL] L, FX, Tim S A (50417425295, 1009131726...",19,1009131726,Q7W,62M5


In [14]:
bw = sorted(df.BW.unique().tolist())
bw

['',
 '100H',
 '100K',
 '100M',
 '10K0',
 '10K5',
 '10K9',
 '10M0',
 '10M5',
 '10M6',
 '11H0',
 '11K0',
 '11K1',
 '11K2',
 '11K3',
 '11K5',
 '11M0',
 '11M2',
 '11M6',
 '11M8',
 '125K',
 '125M',
 '12K0',
 '12K1',
 '12K2',
 '12K3',
 '12K4',
 '12K5',
 '12K9',
 '12M3',
 '12M4',
 '12M5',
 '12M8',
 '130K',
 '13K0',
 '13K2',
 '13K4',
 '13M0',
 '13M2',
 '13M3',
 '13M5',
 '13M7',
 '13M8',
 '14K0',
 '14K4',
 '14K8',
 '14M0',
 '14M2',
 '14M5',
 '14M6',
 '14M7',
 '14M9',
 '15K0',
 '15K3',
 '15K5',
 '15M0',
 '160K',
 '160M',
 '16H0',
 '16K0',
 '16K9',
 '16M0',
 '16M3',
 '17K3',
 '17M0',
 '17M5',
 '180K',
 '18G0',
 '18K0',
 '18M0',
 '19K6',
 '1G60',
 '1K00',
 '1K25',
 '1K50',
 '1K60',
 '1K64',
 '1M00',
 '1M03',
 '1M04',
 '1M06',
 '1M08',
 '1M09',
 '1M20',
 '1M25',
 '1M28',
 '1M50',
 '1M70',
 '1M75',
 '200H',
 '200K',
 '200M',
 '20K0',
 '20K9',
 '20M0',
 '20M5',
 '20M6',
 '21G8',
 '21K0',
 '21M0',
 '21M7',
 '21M8',
 '222M',
 '225H',
 '22M2',
 '230K',
 '235K',
 '23M0',
 '23M3',
 '23M6',
 '23M8',
 '23M

In [21]:
import re

pattern = r"^(\d{1,3})([HKMG])(\d{0,2})$"

re.match(pattern, '100H')

<re.Match object; span=(0, 3), match='100'>

In [ ]:
import hdf5storage as hdf

In [ ]:
db = hdf.loadmat(f'{pasta}/AnatelDB.mat')

In [ ]:
db = hdf.read(path='/c/Users/rsilva/db/', filename='AnatelDB.mat')

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'AnatelDB.mat', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt


# ax = sns.countplot(x="Num_Serviço", data=base)

# plt.rcParams["figure.figsize"] = [30, 20]
# plt.rcParams["figure.autolayout"] = True

# for p in ax.patches:
#     ax.annotate('{:.1f}'.format(int(p.get_height())), (p.get_x()+0.05, p.get_height()+0.02))
    
# plt.title(f"Total de Entidades: {base.shape[0]}")
# plt.xlabel("Código Serviço")

# plt.savefig("Stats.png")

In [ ]:
base = read_base(pasta)

In [ ]:
base.head()

,Frequência,Num_Serviço,Status,Classe,Entidade,Fistel,Número_da_Estação,Município,UF,Latitude,Longitude,Validade_RF,Num_Ato,Data_Ato,Fonte,Classe_Emissão,Largura_Emissão
0,0.0280,19,L,OP,FURNAS CENTRAIS ELETRICAS S A,01030052263,1557670,Nova Iguaçu,RJ,-22.662777,-43.476387,2033-08-17,-1,,STEL,J9E,8K00
1,0.0285,19,L,OP,COMPANHIA DE GERAÇÃO E TRANSMISSÃO DE ENERGIA ...,50420217282,1494686,Joinville,SC,-26.292500,-48.887222,2025-08-31,-1,,STEL,R3E,2K50
2,0.0300,19,L,OP,FURNAS CENTRAIS ELETRICAS S A,01030052263,859966,Araporã,MG,-18.410000,-49.099998,2033-08-17,-1,,STEL,J3E,1K00
3,0.0300,19,L,OP,FURNAS CENTRAIS ELETRICAS S A,01030052263,859753,Campinas,SP,-22.774166,-47.004444,2033-08-17,-1,,STEL,J3E,1K00
4,0.0300,19,L,OP,FURNAS CENTRAIS ELETRICAS S A,01030052263,1557823,São Paulo,SP,-23.441668,-46.590832,2033-08-17,-1,,STEL,J3E,1K00


In [ ]:
subset = ['Frequência', 'Num_Serviço', 'Status', 'Classe', 'Fistel', 'Número_da_Estação', 'Latitude', 'Longitude', 'Classe_Emissão', 'Largura_Emissão']
# double = base.duplicated(subset=subset, keep=False)
# base[double]

base.drop_duplicates(subset, keep='first').shape

(881692, 17)

In [ ]:
subset = base.columns[:-2]
# double = base.duplicated(subset=subset, keep='first')
# base[double]

base.drop_duplicates(base.columns[:-2], keep='first').reset_index(drop=True)
# base.shape

,Frequência,Num_Serviço,Status,Classe,Entidade,Fistel,Número_da_Estação,Município,UF,Latitude,Longitude,Validade_RF,Num_Ato,Data_Ato,Fonte,Classe_Emissão,Largura_Emissão
26,0.038,19,L,OP,LIGHT SERVICOS DE ELETRICIDADE S A,01030063621,756830,Piraí,RJ,-22.629444,-43.895832,2028-12-12,-1,,STEL,N0N,1K00
33,0.040,19,L,OP,LIGHT SERVICOS DE ELETRICIDADE S A,01030063621,758124,Petrópolis,RJ,-22.433332,-43.166668,2028-12-12,-1,,STEL,A3E,5K00
37,0.040,19,L,OP,LIGHT SERVICOS DE ELETRICIDADE S A,01030063621,758051,Carmo,RJ,-21.840000,-42.570000,2028-12-12,-1,,STEL,N0N,1K00
58,0.044,19,L,OP,LIGHT SERVICOS DE ELETRICIDADE S A,01030063621,814180,Volta Redonda,RJ,-22.513611,-44.089722,2028-12-12,-1,,STEL,A3E,5K00
62,0.045,19,L,OP,LIGHT SERVICOS DE ELETRICIDADE S A,01030063621,749680,Rio de Janeiro,RJ,-22.966667,-43.233334,2028-12-12,-1,,STEL,A3E,5K00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883461,82125.000,19,L,FX,TIM S A,50417425295,1007302558,Ribeirão das Neves,MG,-19.768250,-44.082527,2032-08-31,-1,,STEL,Q7W,62M5
883467,82125.000,19,L,FX,TIM S A,50417425295,1005319658,Uberlândia,MG,-18.942499,-48.300556,2032-08-31,-1,,STEL,Q7W,62M5
883481,82125.000,19,L,FX,TIM S A,50417425295,1005325984,Araxá,MG,-19.582222,-46.953888,2032-08-31,-1,,STEL,Q7W,62M5
883514,82125.000,19,L,FX,TIM S A,50417425295,1008765390,Aparecida de Goiânia,GO,-16.805166,-49.329556,2032-08-31,-1,,STEL,Q7W,62M5


In [ ]:
base[base.Número_da_Estação == 1005304065]

,Frequência,Num_Serviço,Status,Classe,Entidade,Fistel,Número_da_Estação,Município,UF,Latitude,Longitude,Validade_RF,Num_Ato,Data_Ato,Fonte,Classe_Emissão,Largura_Emissão
744796,8118.32,19,L,FX,TIM S A,50417425295,1005304065,Sabará,MG,-19.897028,-43.807056,2032-08-31,-1,,STEL,D7W,29M7
825221,19343.00,19,L,FX,TIM S A,50417425295,1005304065,Sabará,MG,-19.897028,-43.807056,2032-08-31,-1,,STEL,D7W,55M0
831552,19453.00,19,L,FX,TIM S A,50417425295,1005304065,Sabará,MG,-19.897028,-43.807056,2032-08-31,-1,,STEL,D7W,55M0
865366,23275.00,19,L,FX,TIM S A,50417425295,1005304065,Sabará,MG,-19.897028,-43.807056,2032-08-31,-1,,STEL,D7W,56M0
881466,82125.00,19,L,FX,TIM S A,50417425295,1005304065,Sabará,MG,-19.897028,-43.807056,2032-08-31,-1,,STEL,G7W,2G00
881467,82125.00,19,L,FX,TIM S A,50417425295,1005304065,Sabará,MG,-19.897028,-43.807056,2032-08-31,-1,,STEL,Q7W,62M5


In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted constants.ipynb.
Converted filter.ipynb.
Converted index.ipynb.
Converted queries.ipynb.
